In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier

Data first view

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/stroke.csv')
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data cleaning

In [ ]:
df.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [ ]:
df.columns
df.stroke.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [ ]:
df.gender.value_counts()
df.ever_married.value_counts()
df.work_type.value_counts()

Private          2925
Self-employed     819
children          687
Govt_job          657
Never_worked       22
Name: work_type, dtype: int64

In [ ]:
df.bmi.fillna(df.bmi.mean(),inplace=True)

In [ ]:
df.drop(columns=['id'],inplace=True)

In [ ]:
lst=['gender','ever_married','work_type','Residence_type','smoking_status']
dict1={}
for col in lst:
  dict1[col]=LabelEncoder()
  df[col]=dict1[col].fit_transform(df[col])
dict1

{'gender': LabelEncoder(),
 'ever_married': LabelEncoder(),
 'work_type': LabelEncoder(),
 'Residence_type': LabelEncoder(),
 'smoking_status': LabelEncoder()}

In [ ]:
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


In [ ]:
df.stroke.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

Under sampling

In [ ]:
df_0_under=df[df.stroke==0].sample(249)
df_1_under=df[df.stroke==1]
df_under=pd.concat([df_0_under,df_1_under])

In [ ]:
x=df_under.drop(columns='stroke')
y=df_under.stroke

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
sc=StandardScaler()

In [ ]:
x_train_scaled=sc.fit_transform(x_train)
x_test_scaled=sc.transform(x_test)

# Models

In [ ]:
model_tree=DecisionTreeClassifier(max_depth=20)
model_knn=KNeighborsClassifier(metric='euclidean',weights='uniform',n_neighbors=7)
model_svc=SVC(kernel='linear',C=0.01)
model_bayes=BernoulliNB()

In [ ]:
models=[model_tree,model_knn,model_bayes,model_svc]
accuracy={}
for model in models:
  model.fit(x_train_scaled,y_train)
  y_pred=model.predict(x_test_scaled)
  accuracy[f'{model}_acc']=accuracy_score(y_test,y_pred)
  print(f'Classification_report {model}')
  print(classification_report(y_test,y_pred))

Classification_report DecisionTreeClassifier(max_depth=20)
              precision    recall  f1-score   support

           0       0.64      0.68      0.66        74
           1       0.67      0.63      0.65        76

    accuracy                           0.65       150
   macro avg       0.65      0.65      0.65       150
weighted avg       0.65      0.65      0.65       150

Classification_report KNeighborsClassifier(metric='euclidean', n_neighbors=7)
              precision    recall  f1-score   support

           0       0.71      0.69      0.70        74
           1       0.71      0.72      0.71        76

    accuracy                           0.71       150
   macro avg       0.71      0.71      0.71       150
weighted avg       0.71      0.71      0.71       150

Classification_report BernoulliNB()
              precision    recall  f1-score   support

           0       0.83      0.66      0.74        74
           1       0.73      0.87      0.79        76

    accur

Over Sampling

In [ ]:
df_0_over=df[df.stroke==0]
df_1_over=df[df.stroke==1].sample(4861,replace=True)
df_over=pd.concat([df_0_over,df_1_over])

In [ ]:
x1=df_over.drop(columns='stroke')
y1=df_over.stroke

In [ ]:
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.3,random_state=0)

In [ ]:
x1_train_scaled=sc.transform(x1_train)
x1_test_scaled=sc.transform(x1_test)

In [ ]:
model_tree1=DecisionTreeClassifier(max_depth=20)
model_knn1=KNeighborsClassifier(metric='euclidean',weights='uniform',n_neighbors=7)
model_svc1=SVC(kernel='linear',C=0.01)
model_bayes1=BernoulliNB()

In [ ]:
models=[model_tree1,model_knn1,model_bayes1,model_svc1]
accuracy={}
for model in models:
  model.fit(x1_train_scaled,y1_train)
  y1_pred=model.predict(x1_test_scaled)
  accuracy[f'{model}_acc']=accuracy_score(y1_test,y1_pred)
  print(f'Classification_report {model}')
  print(classification_report(y1_test,y1_pred))

Classification_report DecisionTreeClassifier(max_depth=20)
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1461
           1       0.93      1.00      0.96      1456

    accuracy                           0.96      2917
   macro avg       0.96      0.96      0.96      2917
weighted avg       0.96      0.96      0.96      2917

Classification_report KNeighborsClassifier(metric='euclidean', n_neighbors=7)
              precision    recall  f1-score   support

           0       1.00      0.81      0.90      1461
           1       0.84      1.00      0.91      1456

    accuracy                           0.91      2917
   macro avg       0.92      0.91      0.91      2917
weighted avg       0.92      0.91      0.91      2917

Classification_report BernoulliNB()
              precision    recall  f1-score   support

           0       0.78      0.68      0.73      1461
           1       0.72      0.81      0.76      1456

    accur

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE()

In [ ]:
x2=df.drop(columns='stroke')
y2=df.stroke

In [ ]:
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y2,test_size=0.3)
x2_train,y2_train=oversampler.fit_resample(x2_train,y2_train)

In [ ]:
model_tree2=DecisionTreeClassifier(max_depth=20)
model_knn2=KNeighborsClassifier(metric='euclidean',weights='uniform',n_neighbors=7)
model_svc2=SVC(kernel='linear',C=0.01)
model_bayes2=BernoulliNB()

In [ ]:
models=[model_tree2,model_knn2,model_bayes2,model_svc2]
accuracy={}
for model in models:
  model.fit(x2_train,y2_train)
  y2_pred=model.predict(x2_test)
  accuracy[f'{model}_acc']=accuracy_score(y2_test,y2_pred)
  print(f'Classification_report {model}')
  print(classification_report(y2_test,y2_pred))

Classification_report DecisionTreeClassifier(max_depth=20)
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      1455
           1       0.11      0.23      0.15        78

    accuracy                           0.87      1533
   macro avg       0.53      0.57      0.54      1533
weighted avg       0.91      0.87      0.89      1533

Classification_report KNeighborsClassifier(metric='euclidean', n_neighbors=7)
              precision    recall  f1-score   support

           0       0.98      0.79      0.87      1455
           1       0.14      0.67      0.24        78

    accuracy                           0.78      1533
   macro avg       0.56      0.73      0.56      1533
weighted avg       0.94      0.78      0.84      1533

Classification_report BernoulliNB()
              precision    recall  f1-score   support

           0       0.95      0.60      0.74      1455
           1       0.04      0.35      0.08        78

    accur